In [3]:
pow(3, 5) * 5

1215

In [64]:
# 以行为状态进行考虑

class Solution:
    def getMaxGridHappiness(self, m: int, n: int, introvertsCount: int, extrovertsCount: int) -> int:
        
        def decode(state):
            # 解码：每一个格子中住的是内向的人还是外向的人
            vals = [0] * n 
            i = 0
            while state:
                vals[i] = state % 3
                state //= 3
                i += 1
            return vals
        
        def canLive(vals): 
            # 当前这个状态是否是合法的
            introver, extrover = 0, 0
            for v in vals:
                if v == 1:
                    introver += 1
                elif v == 2:
                    extrover += 1
            return introver <= introvertsCount and extrover <= extrovertsCount
        
        def calcHappiness(vals):
            # 计算当前坐法，能一共获得多少幸福感
            total = 0
            for i, v in enumerate(vals):
                total += p2int[v]
                if v == 1:
                    # 内向的人，左右有人的情况下需要减去 30,
                    if (i < n-1 and vals[i+1] != 0) or (i > 0 and vals[i-1] != 0):
                        total -= 30
                elif v == 2:
                    # 外向的人，左右有人的情况下需要 +20,
                    if (i < n-1 and vals[i+1] != 0) or (i > 0 and vals[i-1] != 0):
                        total += 20
            return total
        
        def calcCrossHappiness(pre, cur):
            # 计算两行之间的幸福感
            pre_vals, cur_vals = decode(pre), decode(cur)
            total = calcHappiness(cur_vals) # 不考虑行与行之间的话，幸福感有多少
            for i, v in enumerate(cur_vals):
                if v == 1 and pre_vals[i] != 0:
                    if pre_vals[i] == 1:
                        total -= 30 * 2
                    else:
                        total -= 10
                elif v == 2 and pre_vals[i] != 0:
                    if pre_vals[i] == 1:
                        total -= 10
                    else:
                        total += 20 * 2
            return total
        
        p2int = {0: 0, 1: 120, 2: 40} # 0:不住人，1：内向人，2：外向人
        k = pow(3, n) # 由于每一行的格子中可以是：不放人、外向人、内向人三种
        dp = [-float('inf')] * k # dp[state] 表示不同的状态下 最大的幸福感为多少
        for state in range(k):
            vals = decode(state)
            if canLive(vals): # 当前状态是合法的
                dp[state] = calcHappiness(vals)
        
        for i in range(1, m):
            new_dp = [-float('inf')] * k
            
            for state in range(k):
                vals = decode(state)
                if not canLive(vals):
                    continue
                    
                subset = state
                while subset:
                    # 对于 state 有一个约束条件，就是住的人的数目，内向人 <= introvertsCount, 外向人 <= extroversCount
                    new_dp[state] = max(new_dp[state], dp[state-subset] + calcCrossHappiness(state-subset, subset))
                    subset = (subset - 1) & state
            dp = new_dp
        
        return max(dp)

In [123]:
from functools import lru_cache

class Solution:
    def getMaxGridHappiness(self, m: int, n: int, introvertsCount: int, extrovertsCount: int) -> int:
        
        @lru_cache(None)
        def calcHappiness(values, person_type, col_idx):
            up = 0
            if values[col_idx] == 'i': # 上面有可能是哪些人
                up -= 30
            elif values[col_idx] == 'e':
                up += 20
            
            left = 0
            if col_idx > 0:
                if values[col_idx-1] == 'i': # 左边有可能是哪些人
                    left -= 30
                elif values[col_idx-1] == 'e':
                    left += 20
            
            nebs = (1 if up else 0) + (1 if left else 0)
            ans = 0
            if person_type == 'i':
                ans =  120 - 30 * nebs + up + left
            elif person_type == 'e':
                ans = 40 + 20 * nebs + up + left
            return ans
        
        
        @lru_cache(None)
        def dp(pos, intro, extro, last_values):
            if pos == m*n:
                return 0
            
            ans = 0
            
            list_last_values = list(last_values)
            col_idx = pos % n # 当前位置在那一列
            pre_val = list_last_values[col_idx] # 此时这一列上一轮的值是什么
            
            # 可以不放人
            list_last_values[col_idx] = ''
            ans = dp(pos+1, intro, extro, tuple(list_last_values))
            list_last_values[col_idx] = pre_val
            
            if intro:
                # 可以放内向的人
                cur_happiness = calcHappiness(last_values, 'i', col_idx)
                list_last_values[col_idx] = 'i'
                ans = max(ans, cur_happiness + dp(pos+1, intro-1, extro, tuple(list_last_values)))
                list_last_values[col_idx] = pre_val
            
            if extro:
                # 可以放外向的人
                cur_happiness = calcHappiness(last_values, 'e', col_idx)
                list_last_values[col_idx] = 'e'
                ans = max(ans, cur_happiness + dp(pos+1, intro, extro-1, tuple(list_last_values)))
                list_last_values[col_idx] = pre_val
            
            return ans
        
        return dp(0, introvertsCount, extrovertsCount, tuple([''] * n))

In [124]:
solution = Solution()
solution.getMaxGridHappiness(2, 2, 4, 0)

240

In [107]:
from functools import lru_cache

class Solution:
    def getMaxGridHappiness(self, m: int, n: int, introvertsCount: int, extrovertsCount: int) -> int:
        def get_happiness(j, types, last_values):
            up = last_values[j]
            left = last_values[j-1] if j > 0 else ''
            
            up_diff = 0
            if up == 'i':
                up_diff = -30
            elif up == 'e':
                up_diff = 20
            
            left_diff = 0
            if left == 'i':
                left_diff = -30
            elif left == 'e':
                left_diff = 20
            
            neb = (1 if up else 0) + (1 if left  else 0)
            ans = 0
            if types == 'i':
                ans = 120 - 30 * neb + left_diff + up_diff
            elif types == 'e':
                ans = 40 + 20 * neb + left_diff + up_diff
            return ans
        
        @lru_cache(None)
        def dp(pos, intro, extro, last_values):
            if pos == m * n:
                return 0
            
            j = pos % n # 所在列
            list_last_values = list(last_values)
            prev_value = list_last_values[j]
            
            # 当前位置不住人
            list_last_values[j] = ''
            ans = dp(pos + 1, intro, extro, tuple(list_last_values))
            list_last_values[j] = prev_value
            
            # 住的是内向的人
            if intro:
                happiness = get_happiness(j, 'i', last_values)
                print(last_values, happiness)
                list_last_values[j] = 'i'
                ans = max(ans, happiness + dp(pos+1, intro-1, extro, tuple(list_last_values)))
                list_last_values[j] = prev_value
            
            # 住的是外向的人
            if extro:
                happiness = get_happiness(j, 'e', last_values)
                print(happiness)
                list_last_values[j] = 'e'
                ans = max(ans, happiness + dp(pos+1, intro, extro-1, tuple(list_last_values)))
                list_last_values[j] = prev_value
            return ans
        
        return dp(0, introvertsCount, extrovertsCount, tuple([''] * n))

In [108]:
solution = Solution()
solution.getMaxGridHappiness(2, 2, 4, 0)

('', '') 120
('', '') 120
('i', '') 60
('', '') 120
('', 'i') 60
('', 'i') 120
('i', 'i') 0
('', '') 120
('', '') 120
('i', '') 60
('i', '') 60
('i', '') 60
('', 'i') 60
('i', 'i') 60
('i', 'i') 0


240